Same installation procedure as in lab 03

In [1]:
# Collegamento a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#funzione per contare le immagini nelle cartelle
import os
def count_image_folders(main_folder):
    """
    Conta il numero di immagini in ogni sottocartella contenente immagini e ignora le cartelle con 'mask' nel nome.
    """
    image_extensions = ('.png', '.jpg', '.jpeg')  # Estensioni valide per immagini
    total_images = 0  # Variabile per il totale complessivo delle immagini

    for subfolder in os.listdir(main_folder):
        subfolder_path = os.path.join(main_folder, subfolder)

        if os.path.isdir(subfolder_path):  # Controlla se è una directory
            for inner_folder in os.listdir(subfolder_path):
                inner_folder_path = os.path.join(subfolder_path, inner_folder)

                # Escludi le cartelle che contengono "mask" nel nome
                if os.path.isdir(inner_folder_path) and "images" not in inner_folder.lower():
                    # Conta il numero di immagini nella sottocartella
                    image_count = sum(
                        1 for filename in os.listdir(inner_folder_path)
                        if os.path.isfile(os.path.join(inner_folder_path, filename)) and filename.endswith(image_extensions)
                    )

                    if image_count > 0:  # Se ci sono immagini
                        print(f"La sottocartella '{inner_folder_path}' contiene {image_count} immagini.")
                        total_images += image_count  # Aggiungi al totale complessivo

    print(f"Numero totale di immagini: {total_images}")
    return total_images

count_image_folders('/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_DATASET_INFERENCE/TRAIN')

La sottocartella '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_DATASET_INFERENCE/TRAIN/seq3/masks' contiene 15 immagini.
La sottocartella '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_DATASET_INFERENCE/TRAIN/seq1/masks' contiene 36 immagini.
La sottocartella '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_DATASET_INFERENCE/TRAIN/seq4/masks' contiene 48 immagini.
La sottocartella '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_DATASET_INFERENCE/TRAIN/seq22/masks' contiene 46 immagini.
La sottocartella '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_DATASET_INFERENCE/TRAIN/seq16/masks' contiene 40 immagini.
La sottocartella '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_DATASET_INFERENCE/TRAIN/seq7/masks' contiene 48 immagini.
La sottocartella '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_DATASET_INFERENCE/TRAIN/seq20/masks' contiene 52 immagi

2631

In [2]:
!pip install -q torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install -q mmcv==2.0.0rc4 -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 949.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 75.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.14.0 requires torch>=1.13.0, but you have torch 1.11.0+cu113 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 19.7 MB/s eta 0:00:00


In [3]:
!pip install -q mmsegmentation==1.2.2
!pip install -q mmengine==0.10.3
!pip install -q ftfy==6.2.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.9 MB/s eta 0:00:00


In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.11.0+cu113 True
1.2.2


In [5]:
import os
import numpy as np
import torch
import json

from mmseg.apis import init_model, inference_model

from PIL import Image
from tqdm import tqdm

from mmengine import Config

from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset

from skimage import morphology, measure

import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:

import os
from PIL import Image

def reorganize_test_data():
    main_folder = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_DATASET_INFERENCE/VALID'
    output_folder = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_SOLO_VALID_riorg'

    # Percorsi di destinazione per immagini e maschere
    dest_images_folder = os.path.join(output_folder, 'images')
    dest_masks_folder = os.path.join(output_folder, 'masks')

    # Crea le cartelle di destinazione se non esistono
    os.makedirs(dest_images_folder, exist_ok=True)
    os.makedirs(dest_masks_folder, exist_ok=True)

    # Processa ciascuna sequenza
    for seq in ['seq11', 'seq14', 'seq6']:
        # Percorsi delle cartelle di origine
        src_images_folder = os.path.join(main_folder, seq, 'images')
        src_masks_folder = os.path.join(main_folder, seq, 'masks')

        # Percorsi delle cartelle di destinazione per ogni sequenza
        dest_images_seq_folder = os.path.join(dest_images_folder, seq)
        dest_masks_seq_folder = os.path.join(dest_masks_folder, seq)

        # Crea le cartelle di destinazione per la sequenza
        os.makedirs(dest_images_seq_folder, exist_ok=True)
        os.makedirs(dest_masks_seq_folder, exist_ok=True)

        # Sposta le immagini nella nuova struttura
        for image_name in os.listdir(src_images_folder):
            src_image_path = os.path.join(src_images_folder, image_name)
            dest_image_path = os.path.join(dest_images_seq_folder, image_name)
            image=Image.open(src_image_path)
            image.save(dest_image_path)


        # Sposta le maschere nella nuova struttura
        for mask_name in os.listdir(src_masks_folder):
            src_mask_path = os.path.join(src_masks_folder, mask_name)
            dest_mask_path = os.path.join(dest_masks_seq_folder, mask_name)
            mask=Image.open(src_mask_path)
            mask.save(dest_mask_path)

        print(f"Riorganizzata la sequenza {seq}")


    print("Riorganizzazione completata.")

# Esegui la funzione
reorganize_test_data()

Riorganizzata la sequenza seq11
Riorganizzata la sequenza seq14
Riorganizzata la sequenza seq6
Riorganizzazione completata.


In [ ]:
import json
import os

# Percorso del file JSON e della cartella con i file .pth
file_path = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/vis_data/scalars.json'
pth_directory = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri'  # Cambia con il percorso della tua cartella

# Lista per memorizzare le iterazioni con il campo mIoU
miou_entries = []

# Leggi il file JSON
with open(file_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        # Controlla se la chiave 'mIoU' è presente
        if 'mIoU' in data:
            miou_entries.append(data)

# Ordina le iterazioni in base al valore di mIoU (decrescente)
top_miou = sorted(miou_entries, key=lambda x: x['mIoU'], reverse=True)[:5]

# Crea un vettore con i nomi dei file
file_names = [f"iter_{entry['step']}.pth" for entry in top_miou]

# Elimina tutti i file .pth nella cartella esclusi i file nel vettore file_names
for file in os.listdir(pth_directory):
    if file.endswith('.pth') and file not in file_names:
        file_path_to_delete = os.path.join(pth_directory, file)
        os.remove(file_path_to_delete)
        print(f"Eliminato: {file_path_to_delete}")

# Stampa i file che sono stati mantenuti
print("\nFile mantenuti:")
print(file_names)

Eliminato: /content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/iter_329.pth
Eliminato: /content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/iter_658.pth
Eliminato: /content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/iter_987.pth
Eliminato: /content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/iter_1316.pth
Eliminato: /content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/iter_1645.pth
Eliminato: /content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/iter_1974.pth
Eliminato: /content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/iter_2632.pth
Eliminato: /content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/iter_4277.pth
Eliminato: /content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/

In [ ]:
#codice per contare gli elementi in una cartella
all_train = os.listdir('/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_SOLO_TEST_riorg/masks/seq2')

# Conta il numero di elementi
total_train = len(all_train)

print(f"Numero totale di elementi nella cartella con resize: {total_train}")

Numero totale di elementi nella cartella con resize: 0


In [6]:
####################################
#### DO NOT CHANGE THIS CELL #######
####################################
import os
working_folder = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA'
checkpoint_path = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri'

# Results folder
results_folder  = "/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/results_Cambio_parametri_dice_loss_VALID"

# Data folders
test_img_folder = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_SOLO_VALID_riorg/images'
gt_mask_folder  = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/PolypGen-EIM-24-25_SOLO_VALID_riorg/masks'

if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [7]:
####################################
#### DO NOT CHANGE THIS CELL #######
####################################

# Classes and palette for VERSE dataset
classes = ('background', 'object')
palettePOLYP = [
    (0, 0, 0),       # background - black
    (255, 255, 255)  # object - white
]

@DATASETS.register_module()
class POLYP(BaseSegDataset):
    METAINFO = dict(classes=classes, palette=palettePOLYP)

    def __init__(self, **kwargs):
        super().__init__(img_suffix='.png',
                         seg_map_suffix='.png',
                         reduce_zero_label=False,
                         **kwargs)

In [11]:
import numpy as np
from tqdm import tqdm
from PIL import Image
import json
import re
from skimage import morphology, measure
import cv2
from skimage.color import rgb2gray
from scipy import ndimage
from skimage import color, img_as_ubyte
import os
import torch
import json
from skimage import morphology, measure, color, img_as_ubyte, img_as_float
from scipy.ndimage import label
import matplotlib.pyplot as plt



# Funzione per calcolare il Dice coefficient (dal codice 1)
def calculate_2d_dice(pred_image, gt_image):
    """Calculate the 2D Dice coefficient between two images."""
    # Flatten the images to 1D arrays
    pred_flat = pred_image.flatten()
    gt_flat = gt_image.flatten()


    # Handle the case where both masks are empty
    if np.sum(pred_flat) == 0 and np.sum(gt_flat) == 0:
        return 1.0  # Consider it a perfect match (both are empty)

    # Compute the intersection between the prediction and the ground truth
    intersection = np.sum(pred_flat * gt_flat)

    # Calculate the Dice coefficient
    dice = (2. * intersection) / (np.sum(pred_flat) + np.sum(gt_flat) + 1e-7)

    return dice

    # Funzione per estrarre l'ultimo numero dal nome del file
def extract_last_number_from_filename(filename):
    """Estrae l'ultimo numero nel nome del file."""
    # Trova tutti i numeri nel nome del file
    numbers = re.findall(r'\d+', filename)
    # Restituisce l'ultimo numero trovato, se esiste, altrimenti 0
    return int(numbers[-1]) if numbers else 0

def count_connected_objects(volume):
    """Count the number of connected components in a binary volume."""
    labeled_volume = measure.label(volume, connectivity=1)
    num_objects = np.max(labeled_volume)
    return num_objects

def calculate_absolute_error(pred_count, gt_count):
    """Calculate the absolute error in counting connected objects."""
    error = abs(gt_count - pred_count)
    return error


# Funzione per calcolare il Temporal Stability Index (TSI)
def calculate_tsi(mask1, mask2):
    """Calculate the Temporal Stability Index (TSI) between two binary masks."""
    # Converte le maschere in formato binario e tipo float64
    mask1 = np.array(mask1, dtype=np.float64)
    mask2 = np.array(mask2, dtype=np.float64)
    area1 = np.sum(mask1)
    area2 = np.sum(mask2)

    # Evita divisioni per zero
    if area1 + area2 == 0:
        return 0.0

    # Calcola il TSI basato sulla variazione dell'area
    tsi = abs(area1 - area2) / (area1 + area2)
    return tsi

def detect_and_remove_specular_highlights(image):

    image_array = np.array(image)
    gray=rgb2gray(image_array)
    gray=(gray * 255).astype(np.uint8)
    _, mask = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.dilate(mask, kernel, iterations=2)
    corrected_image = cv2.inpaint(image_array, mask, 5, cv2.INPAINT_TELEA)

    return corrected_image, mask

def regional_masking_and_inpainting(img, discontinuity_threshold=1 * 10**(-2)):

        # Converti l'immagine in array numpy
        img_array = np.array(img)

        # Converte in toni di grigio per calcolare i gradienti
        gray_image = color.rgb2gray(img_array)
        J = np.array(gray_image)

        # Definire il kernel derivativo base
        hx = np.array([[-1, 0, 1]])
        hy = np.array([[-1], [0], [1]])

        # Ricavare i gradienti orizzontale e verticale
        Jx = ndimage.correlate(J, hx)
        Jy = ndimage.correlate(J, hy)

        # Identifica la riga centrale
        central_row = Jx[Jx.shape[0] // 2, :]

        # Calcola le differenze tra pixel adiacenti
        diff = np.abs(np.diff(central_row))

        # Trova il primo indice con discontinuità (da sinistra)
        left_bound = np.argmax(diff > discontinuity_threshold)
        # Trova il primo indice con discontinuità (da destra)
        right_bound = len(diff) - np.argmax(diff[::-1] > discontinuity_threshold) - 1

        # Applica la modifica dei pixel sull'immagine a colori
        modified_image = img_array.copy()
        modified_image[:, :left_bound, :] = 16  # Imposta a 16 i pixel a sinistra di left_bound
        modified_image[:, right_bound:, :] = 16  # Imposta a 16 i pixel a destra di right_bound

        # Reset a zero i pixel con tutti i canali pari a 16
        reset_image = modified_image.copy()
        mask = (reset_image[:, :, 0] == 16) & (reset_image[:, :, 1] == 16) & (reset_image[:, :, 2] == 16)
        reset_image[mask] = [0, 0, 0]  # Imposta a [0, 0, 0] i pixel corrispondenti

        # Esegui l'inpainting sull'immagine croppata
        corrected_image, specular_mask = detect_and_remove_specular_highlights(reset_image)


        return corrected_image

def normalizza_immagine(img_path):
    # Read grayscale image
    image = Image.open(img_path)
    image_array=np.array(image)
    # Normalize image
    normalized_image = cv2.normalize(
      image_array, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    return Image.fromarray(normalized_image)

# Funzione per rimuovere regioni connesse minori di una soglia
def remove_small_regions(mask, min_size):
    # Identifica le regioni connesse
    labeled_mask, num_features = label(mask)
    # Crea una maschera vuota
    filtered_mask = np.zeros_like(mask, dtype=np.uint8)
    # Mantieni solo le regioni connesse maggiori della soglia
    for region_label in range(1, num_features + 1):
        region = (labeled_mask == region_label)
        if np.sum(region) >= min_size:
            filtered_mask[region] = 1
    return filtered_mask

# Funzione per aggiungere un polipo basandosi sulle maschere adiacenti
def add_polyp_between_adjacent_frames(prev_mask, curr_mask, next_mask):
    if np.max(curr_mask) == 0 and np.max(prev_mask) > 0 and np.max(next_mask) > 0:
        intersection_mask = np.logical_and(prev_mask > 0, next_mask > 0)
        if np.any(intersection_mask):
            curr_mask[intersection_mask] = 1
    return curr_mask

# Funzione per elaborare tutte le maschere di una sequenza
def process_masks_in_sequence(results_folder,seq,sequence_images, output_folder, percentuale=1):
    if len(sequence_images) < 3:
        print(f"Sequenza troppo corta ({len(sequence_images)} immagini) per essere elaborata.")
        return

    for i in tqdm(range(len(sequence_images)), desc="Processing masks", unit="mask"):
        try:
            # Percorso della maschera corrente
            curr_path = sequence_images[i]


            # Ricava solo la base
            base = os.path.basename(curr_path)

             # Carica la maschera corrente
            curr_mask = np.array(Image.open(os.path.join(results_folder,seq,base.replace('.jpg','.png'))))

            # Calcola l'area minima per la rimozione delle piccole regioni
            height, width = curr_mask.shape[:2]
            min_size = (height * width) * (percentuale / 100)

            if i == 0 or i == len(sequence_images) - 1:
                # Prima o ultima immagine: solo rimozione di piccole regioni
                continue
            else:
                # Immagini centrali: rimozione piccole regioni + aggiunta polipi
                base_i_meno_1 = os.path.basename(sequence_images[i - 1])
                prev_mask = np.array(Image.open(os.path.join(results_folder,seq,base_i_meno_1.replace('.jpg','.png'))))
                base_i_piu_1 = os.path.basename(sequence_images[i + 1])
                next_mask = np.array(Image.open(os.path.join(results_folder,seq,base_i_piu_1.replace('.jpg','.png'))))

                processed_mask = add_polyp_between_adjacent_frames(prev_mask, curr_mask, next_mask)

            # Salva la maschera aggiornata
            relative_path = os.path.join(seq, base.replace('.jpg','.png'))
            output_path = os.path.join(output_folder, relative_path)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            Image.fromarray(processed_mask).save(output_path)
        except Exception as e:
            print(f"Errore durante l'elaborazione dell'immagine: {curr_path}")
            print(f"Dettagli dell'errore: {e}")


In [12]:
####################################
#### DO NOT CHANGE THIS CELL #######
####################################

from mmengine.config import Config
from skimage import img_as_float, img_as_ubyte
pth_directory = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri'  # Cambia con il percorso della tua cartella

config_file = os.path.join(checkpoint_path,'unet-s5-d16_fcn_4xb4-160k_cityscapes-512x1024.py')
cfg = Config.fromfile(config_file)
cfg.device='cuda'

iterazioni_migliori=[f for f in os.listdir(pth_directory) if f.endswith(".pth")]
# Visualizzazione del config caricato (unet)

for iterazione in iterazioni_migliori:
  #print(f'Config:\n{cfg.pretty_text}')

  checkpoint_path1=os.path.join(checkpoint_path, iterazione)
  # Initialize the model from the config and the checkpoint
  model = init_model(cfg, checkpoint_path1, 'cuda:0')

  #prima erano due parti divise
  # Ottieni la lista delle sequenze (es. seq12, seq13, seq18)
  seq_folders = os.listdir(test_img_folder)

  # Crea un dizionario per memorizzare le immagini per ogni sequenza
  seq_dict = {}

  # Itera attraverso le cartelle delle sequenze
  for seq in seq_folders:
      seq_images_folder = os.path.join(test_img_folder, seq)

      # Verifica che la cartella esista e contenga immagini
      if os.path.isdir(seq_images_folder):
          # Ottieni la lista delle immagini in quella sequenza
          img_list = os.listdir(seq_images_folder)

          # Verifica che la lista di immagini non sia vuota
          if img_list:
              for img_name in img_list:
                  # Assicurati che l'immagine sia un file e non una sottocartella
                  if os.path.isfile(os.path.join(seq_images_folder, img_name)):
                      # Aggiungi la sequenza nel dizionario se non è già presente
                      if seq not in seq_dict:
                          seq_dict[seq] = []

                      # Aggiungi l'immagine alla lista della sequenza
                      seq_dict[seq].append(os.path.join(seq_images_folder, img_name))

  # Ordinamento delle immagini per ogni sequenza
  for seq in seq_dict:
      seq_dict[seq].sort()

  # Ora seq_dict conterrà la lista delle immagini per ciascuna sequenza
  # NB
  #
  # Here you can change everything, just make sure that you write one mask for each image of the subject in the correct format.
  # You need to create one folder for each subject and save the predicted masks in that folder.
  #
  # Tips:
  #       - add some checks on the number of items you create, make sure you have one folder for each subject and all the masks for that subject have been written
  #       - images have different shapes, some models might require dimensions being multiple of 16 or 32, use padding and resizing with caution. Make sure to save the mask in the original dimensions.
  #       - add checks to make sure masks are in the 0-255 format and not 0-1


  ###############################
  ####### Inference Loop ########
  ###############################
  new_size=(256,256)
  discontinuity_threshold=1 * 10**(-2)
  percentuale=0.6
  # Loop over the subjects and their images
  for subject, img_list in seq_dict.items():
      # Create a folder for each subject in the results folder
      subject_folder = os.path.join(results_folder,iterazione,subject)
      if not os.path.exists(subject_folder):
          os.makedirs(subject_folder)

      ### NB: if you want to apply preprocessing on the 3D volume you need to reconstruct
      # it here and change the loop to cycle over the slices of the volume instead of
      # cycling over the images

      for img_name in tqdm(img_list, desc=f'Processing {subject}'):
          if img_name.endswith('.jpg'):
              img_path = os.path.join(test_img_folder, img_name)

              ###############################
              ##### 2D Pre-Processing #######
              ###############################


              # Carica l'immagine a colori (RGB)
              # Carica l'immagine a colori (RGB)
              image = Image.open(img_path)
              if image is None:
                print(f"Errore nel caricamento dell'immagine: {img_path}")



              dimensioni=np.shape(image)
              # Ridimensiona l'immagine risultante a 256x256
              image1=image.resize(new_size)
              corrected_image=regional_masking_and_inpainting(image1, discontinuity_threshold)

              corrected_image_pil = Image.fromarray(img_as_ubyte(corrected_image))



              # Save img to temporary file
              corrected_image_pil.save('tmp_img.png')

              image_modified=normalizza_immagine('tmp_img.png')

              image_modified.save('tmp_img.png')




              ###############################
              #######    Inference   ########
              ###############################


              result = inference_model(model, 'tmp_img.png')

              # Get data from the result
              pred_label = result.pred_sem_seg.data.squeeze()
              pred_label = pred_label.cpu().numpy().astype(np.uint8)
              height, width = pred_label.shape[:2]  # Ottieni altezza e larghezza della maschera
              min_size = (height * width) * (percentuale / 100)  # Calcola l'area minima

              ###############################
              ###### 2D Post-Processing #####
              ###############################

              # Filtra le regioni connesse
              filtered_mask = remove_small_regions(pred_label, min_size=min_size)



              ###############################
              #######   Save results  #######
              ###############################




              # Save the result
              pred_label_img = Image.fromarray(filtered_mask, mode='L')
              #pred_label_img = Image.fromarray(pred_label, mode='L')
              pred_label_img = pred_label_img.resize((dimensioni[1],dimensioni[0]))
              pred_label_img.save(os.path.join(subject_folder, os.path.basename(img_name).replace(".jpg",".png")))

  # Imposta i percorsi
  results_folder = results_folder
  output_folder = results_folder
  os.makedirs(output_folder, exist_ok=True)  # Crea la cartella se non esiste

  # Itera su tutte le sequenze nel dizionario
  for seq, sequence_images in seq_dict.items():
      print(f"Processing sequence: {seq}")
      process_masks_in_sequence(os.path.join(results_folder,iterazione),seq,sequence_images, output_folder, percentuale=2)



  # Lista delle sequenze (sub-cartelle)
  sequences = os.listdir(gt_mask_folder)

  # Dizionario per salvare i risultati per sequenza
  evaluation_results = {}

  # Calcola le metriche per tutte le immagini
  for seq in sequences:
    # Costruisci i percorsi per le sottocartelle
    gt_seq_folder = os.path.join(gt_mask_folder, seq)
    result_seq_folder = os.path.join(results_folder, iterazione, seq)

    # Ordinare le liste di maschere manuali e automatiche
    list_mask_manual = sorted(os.listdir(gt_seq_folder), key=extract_last_number_from_filename)
    list_mask_auto = sorted(os.listdir(result_seq_folder), key=extract_last_number_from_filename)

    # Assicurati che la lista delle maschere sia la stessa in entrambe le cartelle
    if len(list_mask_manual) != len(list_mask_auto):
        raise ValueError(f"Il numero di maschere nelle cartelle {gt_seq_folder} e {result_seq_folder} non corrispondono!")

    # Liste per raccogliere le metriche per questa sequenza
    Dices = []
    absolute_errors = []
    TSIs = []  # Lista per raccogliere i valori di TSI

    # Lista per raccogliere le metriche per ogni immagine
    images_metrics = []

    # Ciclo su ogni immagine e calcola le metriche
    prev_auto_mask = None  # Maschera automatica precedente per il calcolo del TSI
    for manual_mask_path, auto_mask_path in tqdm(zip(list_mask_manual, list_mask_auto)):
        # Rimuoviamo "_mask" dal nome del file per fare il confronto
        if "_mask" in manual_mask_path:
            manual_mask_name = manual_mask_path.replace("_mask", "")
        else:
            manual_mask_name = manual_mask_path

        # Verifica che i nomi corrispondano tra ground truth e risultato
        if manual_mask_name.replace(".jpg","") != auto_mask_path.replace(".png",""):
            raise ValueError(f"Il nome della maschera nel ground truth ({manual_mask_path}) non corrisponde con il nome nel risultato ({auto_mask_path})!")

        # Lettura delle maschere
        manual_mask = Image.open(os.path.join(gt_seq_folder, manual_mask_path))
        auto_mask = Image.open(os.path.join(result_seq_folder, auto_mask_path))

        auto_mask = np.array(auto_mask)
        manual_mask = np.array(manual_mask.convert('L'))
        manual_mask = np.where(manual_mask > 127, 1, 0).astype(np.uint8)

        # Calcola il Dice coefficient
        Dice = calculate_2d_dice(manual_mask, auto_mask)

        # Aggiungi il Dice alla lista
        Dices.append(Dice)

        # Conta gli oggetti connessi
        pred_count = count_connected_objects(auto_mask)
        gt_count = count_connected_objects(manual_mask)

        # Calcola l'errore assoluto
        absolute_error = calculate_absolute_error(pred_count, gt_count)
        absolute_errors.append(absolute_error)

        # Calcola il TSI se esiste una maschera precedente
        if prev_auto_mask is not None:
            tsi = calculate_tsi(prev_auto_mask, auto_mask)
            TSIs.append(tsi)
        else:
            tsi = None  # Nessun valore TSI per il primo frame

        # Aggiorna la maschera precedente
        prev_auto_mask = auto_mask

        # Aggiungi le metriche di questa immagine alla lista per questa sequenza
        images_metrics.append({
            'image': manual_mask_name,  # Nome dell'immagine
            'Dice': float(Dice),
            'Absolute Error': float(absolute_error),
            'TSI': float(tsi) if tsi is not None else None
        })

    # Calcola media e deviazione standard per le metriche per la sequenza
    evaluation_results[seq] = {
        'metrics': {
            'Dice': {
                'mean': float(np.mean(Dices)),
                'std': float(np.std(Dices))
            },
            'Absolute Error': {
                'mean': float(np.mean(absolute_errors)),
                'std': float(np.std(absolute_errors))
            },
            'TSI': {
                'mean': float(np.mean(TSIs)) if TSIs else None,
                'std': float(np.std(TSIs)) if TSIs else None
            }
        },
        'images': images_metrics  # Aggiungi le metriche per ogni immagine
    }

  # Stampa i risultati
  for seq, metrics in evaluation_results.items():
    print(f"\nRisultati per la sequenza {seq}:")
    for metric, values in metrics['metrics'].items():
        print(f"  {metric} - Media: {values['mean']:.4f}, Deviazione standard: {values['std']:.4f}")

  # Salva i risultati dell'evaluazione su un file JSON
  evaluation_json_path = os.path.join(results_folder, iterazione, 'performance_dice_per_paziente.json')
  with open(evaluation_json_path, 'w') as json_file:
    json.dump(evaluation_results, json_file, indent=4)

  # Elimina il modello dalla memoria GPU
  del model
  torch.cuda.empty_cache()




Loads checkpoint by local backend from path: /content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri/iter_2303.pth


Processing seq6: 100%|██████████| 91/91 [02:04<00:00,  1.37s/it]


Processing sequence: seq11


Processing masks: 100%|██████████| 228/228 [00:06<00:00, 32.92mask/s]


Processing sequence: seq14


...

Processing sequence: seq14


Processing masks: 100%|██████████| 249/249 [00:14<00:00, 17.66mask/s]


Processing sequence: seq6


Processing masks: 100%|██████████| 91/91 [00:06<00:00, 14.15mask/s]
228it [00:05, 44.82it/s]
249it [00:14, 17.47it/s]
91it [00:05, 15.79it/s]



Risultati per la sequenza seq11:
  Dice - Media: 0.4783, Deviazione standard: 0.4487
  Absolute Error - Media: 0.4254, Deviazione standard: 0.5032
  TSI - Media: 0.1532, Deviazione standard: 0.3162

Risultati per la sequenza seq14:
  Dice - Media: 0.6186, Deviazione standard: 0.3874
  Absolute Error - Media: 2.5823, Deviazione standard: 6.2378
  TSI - Media: 0.3132, Deviazione standard: 0.3780

Risultati per la sequenza seq6:
  Dice - Media: 0.4879, Deviazione standard: 0.4687
  Absolute Error - Media: 0.5165, Deviazione standard: 0.5997
  TSI - Media: 0.2602, Deviazione standard: 0.3919


In [13]:
import os
import json
import numpy as np

# Percorsi
pth_directory = '/content/drive/MyDrive/CHALLENGE_NUOVA_CARTELLA/weights_unet_dice_loss_cambio_parametri'

# Lista delle iterazioni migliori
iterazioni_migliori = [f for f in os.listdir(pth_directory) if f.endswith(".pth")]
list_results_folder = os.listdir(results_folder)

# Verifica che le due liste abbiano la stessa lunghezza
for cartella, iterazione in zip(list_results_folder, iterazioni_migliori):
    # Leggi il file JSON
    with open(os.path.join(results_folder, cartella,'performance_dice_per_paziente.json'), 'r') as file:
        data = json.load(file)

    # Inizializza dizionari per salvare i valori di tutte le metriche globalmente
    metrics = {"Dice": [], "Absolute Error": [], "TSI": []}

    # Itera su tutte le sequenze nel file JSON
    for seq_name, seq_data in data.items():
        # Estrarre le immagini e le loro metriche
        images = seq_data['images']

        # Popola i dizionari con i valori per ogni immagine
        for img in images:
            for metric in metrics:
                value = img.get(metric)
                if value is not None:
                    metrics[metric].append(value)

    # Calcola la media e la deviazione standard globali per ciascuna metrica
    overall_stats = {}
    for metric, values in metrics.items():
        if values:  # Verifica che ci siano valori per la metrica
            mean = np.mean(values)
            std = np.std(values)
            overall_stats[metric] = {"mean": mean, "std": std}

    # Stampa i risultati globali
    print("Statistiche globali per", iterazione, ":")
    for metric, stats in overall_stats.items():
        print(f"{metric}: Mean = {stats['mean']:.4f}, Std = {stats['std']:.4f}")


Statistiche globali per iter_2303.pth :
Dice: Mean = 0.4651, Std = 0.4393
Absolute Error: Mean = 1.5335, Std = 4.2042
TSI: Mean = 0.2361, Std = 0.3558
Statistiche globali per iter_2961.pth :
Dice: Mean = 0.5925, Std = 0.4299
Absolute Error: Mean = 1.3504, Std = 4.3112
TSI: Mean = 0.1780, Std = 0.3252
Statistiche globali per iter_3290.pth :
Dice: Mean = 0.5174, Std = 0.4471
Absolute Error: Mean = 1.4278, Std = 4.2636
TSI: Mean = 0.2146, Std = 0.3553
Statistiche globali per iter_3619.pth :
Dice: Mean = 0.5522, Std = 0.4460
Absolute Error: Mean = 1.3891, Std = 4.2611
TSI: Mean = 0.1844, Std = 0.3359
Statistiche globali per iter_3948.pth :
Dice: Mean = 0.5414, Std = 0.4318
Absolute Error: Mean = 1.3856, Std = 4.2820
TSI: Mean = 0.2405, Std = 0.3644
